LSTM

In [ ]:
from datetime import datetime, timedelta
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
acao = 'MGLU3.SA'
periodo_tempo_de_precos_acao = "5y"
ativo = yf.download(acao, period=periodo_tempo_de_precos_acao)

In [ ]:
ativo.head()

In [ ]:
# Pega somente o atributo "Close" (preço de fechamento da ação em cada dia) e armazena em um array bidimensional
# O array precisa ser bidimensional porque o MinMaxScaler espera uma entrada com esse formato.
ativo_close = ativo['Close'].dropna().to_numpy().reshape(-1, 1)

In [ ]:
# Criar um DataFrame com os valores ajustados
data = pd.DataFrame(ativo_close)
data.columns = ['Close']

In [ ]:
# Normalizar os dados para os modelos
scaler = MinMaxScaler(feature_range=(0, 1))
data['Scaled_Close'] = scaler.fit_transform(data[['Close']])

In [ ]:
# Função para criar janelas de dados
# janela de dados = quantidade de dias do X
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)


In [ ]:
window_size = 60
scaled_data = data['Scaled_Close'].values
X, y = create_dataset(scaled_data, window_size)

In [ ]:
X[0] # 60 (window-size) valores que ele usa para prever o valor do y. 

In [ ]:
scaled_data.shape, X.shape, y.shape

In [ ]:
# Dividir os dados em treinamento e teste
# O argumento shuffle=False desativa a aleatorização dos dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
# Ajustar o formato para o LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
len(X_test)
# X = 1184 (5 anos de dias úteis em janelas de 60 dias [71040 valores])
# X_test = 237 dias (20% dos 5 anos de dias úteis em janelas de 60 dias [14220 valores])
# y = 1184 valores reais da ação
# y_test = 20% dos valores reais da ação que serão utilizados para comparmos as nossas previsões

In [ ]:
# Criação de um modelo sequencial, onde as camadas são empilhadas uma após a outra
model = Sequential([
    # Define a forma da entrada: uma sequência de 'window_size' passos, cada um com 1 feature (ex: preço de fechamento da ação)
    Input(shape=(window_size, 1)),
    # Primeira camada LSTM com 100 unidades; retorna a sequência completa (uma saída por passo de tempo)
    LSTM(100, return_sequences=True),
    # Dropout de 30% para evitar overfitting (desativa aleatoriamente 30% dos neurônios durante o treino)
    Dropout(0.3),
    # Segunda camada LSTM com 100 unidades; também retorna a sequência completa
    LSTM(100, return_sequences=True),
    # Outro Dropout de 30% para regularização
    Dropout(0.3),
    # Terceira camada LSTM com 100 unidades; agora retorna apenas a última saída da sequência
    LSTM(100),
    # Mais um Dropout de 30% antes da saída final
    Dropout(0.3),
    # Camada densa (totalmente conectada) com 1 unidade; representa o valor final previsto (ex: o próximo valor)
    Dense(1)
])


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')


In [ ]:
# Treinar o modelo
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)

model.fit(X_train, y_train, 
               epochs=100, 
               batch_size=32, 
               validation_data=(X_test, y_test),
               callbacks=[early_stopping, reduce_lr], 
               verbose=1)


In [ ]:
# Fazer previsões
predicted_prices = model.predict(X_test)

# Inverter a normalização
predicted_prices = scaler.inverse_transform(predicted_prices)

In [ ]:
# Cria um DataFrame com as datas, os valores reais e os valores previstos
df_predictions = pd.DataFrame({
    "Date": ativo.index[len(X_train) + window_size : len(X_train) + window_size + len(predicted_prices)],
    "Real": ativo['Close'].iloc[len(X_train) + window_size : len(X_train) + window_size + len(predicted_prices)].values.flatten(),
    "Previsão": predicted_prices.flatten()
})

# Formata a coluna de data e define como índice
df_predictions["Date"] = pd.to_datetime(df_predictions["Date"])
df_predictions.set_index("Date", inplace=True)


In [ ]:
df_predictions

In [ ]:
# Calcular métricas de desempenho do modelo
mae = mean_absolute_error(df_predictions["Real"], df_predictions["Previsão"])  # Erro absoluto médio
mse = mean_squared_error(df_predictions["Real"], df_predictions["Previsão"])  # Erro quadrático médio
rmse = np.sqrt(mse)  # Raiz do erro quadrático médio
r2 = r2_score(df_predictions["Real"], df_predictions["Previsão"])  # Coeficiente de determinação (R²)

# Imprimir as métricas com comentários para organização
print("📊 MÉTRICAS DE ERRO DO MODELO LSTM 📊\n")

print("🔹 MAE  (Erro Absoluto Médio):")
print(f"     → Em média, o modelo erra cerca de {mae:.4f} unidades em cada previsão.\n")

print("🔹 MSE  (Erro Quadrático Médio):")
print(f"     → Erros maiores são penalizados. Valor: {mse:.4f}\n")

print("🔹 RMSE (Raiz do Erro Médio Quadrático):")
print(f"     → Erro médio em unidades reais: {rmse:.4f}\n")

print("🔹 R²   (Coeficiente de Determinação):")
print(f"     → {r2:.4f} (ou {r2 * 100:.2f}%) da variação nos dados reais foi explicada pelo modelo.\n")

print("✅ Interpretação rápida:")
print("     • Quanto menores o MAE, MSE e RMSE, melhor o modelo.")
print("     • Quanto mais próximo de 1 o R², melhor a qualidade da previsão.")


In [ ]:
import matplotlib.pyplot as plt

# Criar o gráfico
plt.figure(figsize=(12, 6))
plt.plot(df_predictions.index, df_predictions["Real"], label="Preço Real", linestyle="-", color="blue")
plt.plot(df_predictions.index, df_predictions["Previsão"], label="Previsão LSTM", linestyle="--", color="red")

# Adicionar títulos e legendas
plt.title("Comparação entre Preço Real e Previsão do Modelo LSTM", fontsize=14)
plt.xlabel("Data", fontsize=12)
plt.ylabel("Preço de Fechamento", fontsize=12)
plt.legend()
plt.grid(True)

# Mostrar o gráfico
plt.show()


In [ ]:
# Calcular média de acertos e expectativa de lucro
df_predictions = df_predictions.copy()

df_predictions['Variação percentual real'] = df_predictions['Real'].pct_change()

#Nova coluna com a variação percentual da ação prevista
df_predictions['Variação percentual previsão'] = df_predictions['Previsão'].pct_change()

#Remover valores nulos
df_predictions = df_predictions.dropna()

#Nova coluna com a informação se a ação subiu ou não
df_predictions['Ação subiu'] = np.where(df_predictions['Variação percentual real'] > 0, 
                                                      True, False)

#Nova coluna com a informação se a previsão subiu ou não
df_predictions['Previsão subiu'] = np.where(df_predictions['Variação percentual previsão'] > 0, 
                                                      True, False)

#Nova coluna com a informação se o modelo acertou a tendência
df_predictions['Modelo acertou tendência'] = np.where(df_predictions['Ação subiu'] == df_predictions['Previsão subiu']
                                      , True, False)

#Nova coluna com a variação percentual real absoluta
df_predictions['Variação percentual real absoluto'] = df_predictions['Variação percentual real'].abs()

#calcular media de acertos e expectativa de lucro
acertou_lado = df_predictions['Modelo acertou tendência'].sum()/len(df_predictions['Modelo acertou tendência'])

#calcular media de erros e expectativa de lucro
errou_lado = 1 - acertou_lado

#calcular media de lucro agrupado por acertos e erros
media_lucro = df_predictions.groupby('Modelo acertou tendência')['Variação percentual real absoluto'].mean()

#calcular media de acertos e expectativa de lucro
acertou_lado = df_predictions['Modelo acertou tendência'].sum()/len(df_predictions['Modelo acertou tendência'])

#calcular media de erros e expectativa de lucro
errou_lado = 1 - acertou_lado

#calcular media de lucro agrupado por acertos e erros
media_lucro = df_predictions.groupby('Modelo acertou tendência')['Variação percentual real absoluto'].mean()

#calcular expectativa matematica de lucro
exp_mat_lucro = acertou_lado * media_lucro[1] - media_lucro[0] * errou_lado

#calcular ganho sobre perda
ganho_sobre_perda = media_lucro[1]/media_lucro[0]

#calcular rendimento diario
exp_mat_lucro_percent = exp_mat_lucro * 100

print("📊 MÉTRICAS DO MODELO - Último ano 📊\n")
print(f"Media lucro: {media_lucro}")
print(f"Ganho sobre perda: {ganho_sobre_perda}")
print(f"Acertou lado: {acertou_lado}")
print("rendimento diário: " + str(round(exp_mat_lucro_percent, 4)) + "%")


In [ ]:
# Garante que o índice está como datetime
df_predictions.index = pd.to_datetime(df_predictions.index)

# Define a data final (último dia disponível) e inicial (30 dias antes)
data_final = df_predictions.index.max()
data_inicial = data_final - timedelta(days=30)

# Filtra apenas o último mês
ultimo_mes = df_predictions.loc[data_inicial:data_final].copy()

# --- Cálculo das métricas de rendimento diário ---
# Colunas necessárias (valores já como float)
ultimo_mes['Variação percentual real'] = ultimo_mes['Real'].pct_change()
ultimo_mes['Variação percentual previsão'] = ultimo_mes['Previsão'].pct_change()

# Remover valores nulos
ultimo_mes = ultimo_mes.dropna()

# Acertos de tendência
ultimo_mes['Ação subiu'] = ultimo_mes['Variação percentual real'] > 0
ultimo_mes['Previsão subiu'] = ultimo_mes['Variação percentual previsão'] > 0
ultimo_mes['Modelo acertou tendência'] = ultimo_mes['Ação subiu'] == ultimo_mes['Previsão subiu']

# Variação percentual absoluta
ultimo_mes['Variação percentual real absoluto'] = ultimo_mes['Variação percentual real'].abs()

# Probabilidades de acerto e erro
acertou_lado = ultimo_mes['Modelo acertou tendência'].mean()
errou_lado = 1 - acertou_lado

# Média de lucro por acertos/erros
media_lucro = ultimo_mes.groupby('Modelo acertou tendência')['Variação percentual real absoluto'].mean()

# Expectativa matemática de lucro diário
exp_mat_lucro = acertou_lado * media_lucro[True] - errou_lado * media_lucro[False]
exp_mat_lucro_percent = exp_mat_lucro * 100

# Ganho sobre perda
ganho_sobre_perda = media_lucro[True] / media_lucro[False]

# --- Impressão dos resultados ---
print("📊 MÉTRICAS DO MODELO - Último mês 📊\n")
print(f"Media lucro: {media_lucro}")
print(f"Ganho sobre perda: {ganho_sobre_perda:.4f}")
print(f"Acertou lado: {acertou_lado:.4f}")
print(f"Rendimento diário esperado: {exp_mat_lucro_percent:.4f}%")

In [ ]:
# Define o horário atual
#teste
hoje = datetime.now()

# Define se a previsão será para hoje ou amanhã com base no horário
if hoje.hour > 18:
    texto_previsao = "\nPreço de fechamento previsto para amanhã:"
else:
    texto_previsao = "\nPreço de fechamento previsto para hoje:"

# Obtém a última janela usada no treinamento
ultima_janela = X_test[-1]

# Redimensiona para shape (1, window_size, 1) para prever
X_input = ultima_janela.reshape(1, ultima_janela.shape[0], 1)

# Faz a previsão
predicted_price = model.predict(X_input)

# Reverte a escala
predicted_price = scaler.inverse_transform(predicted_price)

# Mostra o resultado
print(texto_previsao)
print(f"R$ {predicted_price[0][0]:.2f}")

In [ ]:
import pandas as pd
from datetime import timedelta

# Define o número de dias para exibir
dias_passados = 4
dias_futuros = 1

# Garante que o DataFrame de previsões está com o índice como datetime
df_predictions.index = pd.to_datetime(df_predictions.index)

# Define a data final (último dia disponível)
data_final = df_predictions.index.max()

# Define o intervalo de datas: do passado até o futuro
data_inicial = data_final - timedelta(days=dias_passados)
data_futura = data_final + timedelta(days=dias_futuros)

# Filtra o DataFrame para o intervalo desejado e apenas as colunas 'Real' e 'Previsão'
df_resultado = df_predictions.loc[data_inicial:data_futura, ['Real', 'Previsão']].copy()

# Formata os valores para 2 casas decimais
df_resultado['Real'] = df_resultado['Real'].map(lambda x: f"R$ {x:.2f}")
df_resultado['Previsão'] = df_resultado['Previsão'].map(lambda x: f"R$ {x:.2f}")

# Renomeia as colunas
df_resultado.columns = ['Preço Real', 'Preço Estimado']

# Exibe o resultado
print("📊 Comparativo de Preço Real vs Estimado:\n")
display(df_resultado)
